In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path = 'Compression Data 14.05.2024.xlsx'
df = pd.read_excel(path)
print(df.head)


In [ ]:
from scipy.signal import find_peaks

# Calculate the gradient between each point
stress = 'Stress (MPa)'
strain = 'Strain'
df['Gradient'] = np.gradient(df[stress], df[strain])
interval = 0.0025


def plot(unit_cell_size, rho, df, peaks):
    # Plot the data
    plt.figure(figsize=(12, 8))
    plt.plot(df['Strain'], df['Stress (MPa)'], label='Stress-Strain Curve')

    # Create a list of (peak index, peak strain, peak stress) tuples
    peak_list = [(i, df['Strain'][i], df['Stress (MPa)'][i]) for i in peaks]
    # Sort peaks by strain for easier processing
    peak_list.sort(key=lambda x: x[1])

    # Group peaks that are within 0.05 of each other and select the highest within each group
    selected_peaks = []
    current_group = []

    for peak in peak_list:
        if not current_group or peak[1] - current_group[-1][1] <= 0.05:
            current_group.append(peak)
        else:
            # Select the peak with the maximum stress from the current group
            highest_peak = max(current_group, key=lambda x: x[2])
            selected_peaks.append(highest_peak)
            current_group = [peak]

    # Don't forget to check the last group
    if current_group:
        highest_peak = max(current_group, key=lambda x: x[2])
        selected_peaks.append(highest_peak)

    # Annotate the selected peaks
    for peak in selected_peaks:
        peak_index, peak_strain, peak_stress = peak
        plt.plot(peak_strain, peak_stress, 'x', label=f'Peak Stress: {peak_stress:.6f}')
        plt.annotate(f'{peak_stress:.2f}', 
                    (peak_strain, peak_stress),
                    textcoords="offset points", 
                    xytext=(0, 10), 
                    ha='center')

    plt.xlabel('Strain')
    plt.ylabel('Stress (MPa)')
    plt.title('Stress-Strain Curve with Stress of Peaks Annotated for '+'Unit Cell Size: '+str(unit_cell_size)+'mm')
    plt.grid(True)
    plt.savefig('S-S-curve-peaks-annotated_'+str(unit_cell_size)+'mm.png')
    plt.show()

peaks, _ = find_peaks(df['Stress (MPa)'])
unit_cell_size_list = [1.428, 1.666, 2.0, 2.5]
rho_list = [0.107, 0.078, 0.054, 0.054]
n = 2.69    
    
for i in range(len(unit_cell_size_list)):
    new_df = df.copy(deep=True)
    new_df['Stress (MPa)'] = new_df['Stress (MPa)'].values*rho_list[i]**n/rho_list[-1]**n
    # df['Strain']
    plot(unit_cell_size_list[i], rho_list[i], new_df, peaks)